In [1]:
import numpy as np
import os
import pandas as pd
import time
from itertools import product
import math

In [ ]:
start = 36000001

end = 39000001


In [ ]:
df_disease_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/9_gold_disease_similarity_whole.tsv")
df_drug_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/9_gold_drug_similarity_whole.tsv")
new_asso = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/10_PREDICT_association.tsv")

pos = new_asso[new_asso.association == 1]
neg = new_asso[new_asso.association == 0]
print len(pos)
print len(neg)

new_asso_li = list()
for index, row in new_asso.iterrows():
    query_drug = row['drug']
    query_disease = row['disease']
    query_asso = row['association']
    new_asso_li.append((query_drug, query_disease, query_asso))
    
new_pos_li = list()
for index2, row2 in pos.iterrows():
    pos_drug = row2['drug']
    pos_disease = row2['disease']
    new_pos_li.append((pos_drug, pos_disease)) 
print 'start : ' + time.strftime('%a %H:%M:%S')
desc_pair = list(product(new_asso_li, new_pos_li))
print 'end : ' + time.strftime('%a %H:%M:%S')

descriptor = pd.DataFrame()
cnt = 0
print 'start : ' + time.strftime('%a %H:%M:%S')
for pair in desc_pair[start:end]:
    query_drug = pair[0][0]
    query_disease = pair[0][1]
    query_asso = pair[0][2]
    pos_drug = pair[1][0]
    pos_disease = pair[1][1]
    if (query_drug == pos_drug) & (query_disease == pos_disease):
            continue
    else:
        found_drug = df_drug_sim[(df_drug_sim.DrugBank_ID1 == query_drug) & (df_drug_sim.DrugBank_ID2 == pos_drug)]
        drug_cheSim = found_drug['CheSim'].values[0]
        drug_pheSim = found_drug['PheSim'].values[0]
    
        found_disease = df_disease_sim[(df_disease_sim.Disease_ID1 == query_disease) & (df_disease_sim.Disease_ID2 == pos_disease)]
        disease_pheSim = found_disease['PheSim'].values[0]
        disease_geSim = found_disease['GeSim'].values[0]
    
        drug_sim = [drug_cheSim, drug_pheSim]
        disease_sim = [disease_geSim, disease_pheSim]
        
        sim = [math.sqrt(x[0]*x[1]) for x in list(product(drug_sim, disease_sim))]
        temp_df = pd.DataFrame(data = [[query_drug, query_disease, query_asso, sim[0], sim[1], sim[2], sim[3]]],
                           columns=['drug', 'disease', 'association', 'DrChe_DiGe', 'DrChe_DiPhe', 'DrPhe_DiGe', 'DrPhe_DiPhe'])
        descriptor = descriptor.append(temp_df)
        cnt = cnt +1
        if cnt % 1000 == 0:
            print "{} : ".format(cnt) + time.strftime('%a %H:%M:%S')
print 'end : ' + time.strftime('%a %H:%M:%S')
print cnt
print descriptor.shape

descriptor.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/5_PREDICT_descriptor/predict_descriptor_{}_{}.tsv".format(start, end-1),
                  sep = '\t', index=False)

1397
105763
start : Mon 23:44:27
end : Mon 23:44:48
start : Mon 23:44:48
1000 : Mon 23:45:06
2000 : Mon 23:45:24
3000 : Mon 23:45:42
4000 : Mon 23:46:00
5000 : Mon 23:46:18
6000 : Mon 23:46:35
7000 : Mon 23:46:55
8000 : Mon 23:47:17
9000 : Mon 23:47:35
10000 : Mon 23:47:55
11000 : Mon 23:48:14
12000 : Mon 23:48:35
13000 : Mon 23:48:56
14000 : Mon 23:49:14
15000 : Mon 23:49:32
16000 : Mon 23:49:53
17000 : Mon 23:50:14
18000 : Mon 23:50:34
19000 : Mon 23:50:57
20000 : Mon 23:51:17
21000 : Mon 23:51:41
22000 : Mon 23:52:01
23000 : Mon 23:52:24
24000 : Mon 23:52:45
25000 : Mon 23:53:07
26000 : Mon 23:53:27
27000 : Mon 23:53:47
28000 : Mon 23:54:09
29000 : Mon 23:54:33
30000 : Mon 23:54:57
31000 : Mon 23:55:16
32000 : Mon 23:55:33
33000 : Mon 23:55:55
34000 : Mon 23:56:15
35000 : Mon 23:56:46
36000 : Mon 23:57:39
37000 : Mon 23:58:24
38000 : Mon 23:59:19
39000 : Tue 00:00:01
40000 : Tue 00:00:20
41000 : Tue 00:00:40
42000 : Tue 00:01:03
43000 : Tue 00:01:26
44000 : Tue 00:01:47
45000 : Tue 

In [ ]:
#### disease similarities 합치는 과정
#disease_ph = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/9_gold_disease_similarity_mimminerc.tsv", index_col=0)
#disease_ge = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/9_gold_disease_similarity_genetic.tsv")
#disease_ph2 = pd.DataFrame()
#for i in disease_ph.columns.values:
#    for j in disease_ph.index.values:
#        temp_df = pd.DataFrame(data = [[int(i), int(j), disease_ph[[i]].loc[j].values[0]]], columns = ['Disease_ID1', 'Disease_ID2', 'Similarity_Score'] )
#        disease_ph2 = disease_ph2.append(temp_df)
#df_disease_sim = pd.merge(left = disease_ph2, right= disease_ge, left_on = ['Disease_ID1', 'Disease_ID2'], right_on = ['Disease_ID1', 'Disease_ID2'])
#df_disease_sim.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/6_new_disease/9_gold_disease_similarity_whole.tsv", sep='\t', index=False)
#df_disease_sim.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/9_gold_disease_similarity_whole.tsv", sep='\t', index=False)
#df_disease_sim = df_disease_sim.rename(columns = {'Similarity_Score_x':'PheSim', 'Similarity_Score_y':'GeSim'})
df_disease_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/9_gold_disease_similarity_whole.tsv")

In [ ]:
#### drug similarities 합치는 과정
#drug_ch = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/9_gold_drug_similarity_structure.tsv")
#drug_ph = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/9_gold_drug_similarity_phenotype.tsv")
#df_drug_sim = pd.merge(left = drug_ch, right= drug_ph, left_on = ['DrugBank_ID1', 'DrugBank_ID2'], right_on = ['DrugBank_ID1', 'DrugBank_ID2'])
#df_drug_sim = df_drug_sim.rename(columns = {'Similarity_Score_x' : 'CheSim', 'Similarity_Score_y':'PheSim'})
#df_drug_sim.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/9_gold_drug_similarity_whole.tsv", index=False, sep='\t')
#df_drug_sim.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/9_gold_drug_similarity_whole.tsv", index=False, sep='\t')
df_drug_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/9_gold_drug_similarity_whole.tsv")

In [ ]:
#### PREDICT용 새로운 association
#asso = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/9_gold_learning_association.tsv")
#new_asso = asso[asso['drug'].isin(df_drug_sim['DrugBank_ID1'].drop_duplicates())]
#new_asso.index = range(len(new_asso))
#new_asso.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/7_new_association/10_PREDICT_association.tsv", sep='\t', index=False)
#new_asso.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/10_PREDICT_association.tsv", sep='\t', index=False)
new_asso = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/10_PREDICT_association.tsv")

In [ ]:
print len(df_drug_sim)
df_drug_sim.head(1)

In [ ]:
print len(df_disease_sim)
df_disease_sim.head(1)

In [ ]:
print len(new_asso)
new_asso.head(1)

In [ ]:
pos = new_asso[new_asso.association == 1]
neg = new_asso[new_asso.association == 0]
print len(pos)
print len(neg)

In [ ]:
new_asso_li = list()
for index, row in new_asso.iterrows():
    query_drug = row['drug']
    query_disease = row['disease']
    query_asso = row['association']
    new_asso_li.append((query_drug, query_disease, query_asso))

In [ ]:
new_pos_li = list()
for index2, row2 in pos.iterrows():
    pos_drug = row2['drug']
    pos_disease = row2['disease']
    new_pos_li.append((pos_drug, pos_disease)) 

In [ ]:
print 'start : ' + time.strftime('%a %H:%M:%S')
desc_pair = list(product(new_asso_li, new_pos_li))
print 'end : ' + time.strftime('%a %H:%M:%S')

In [ ]:
print len(desc_pair)
desc_pair[1395:1399]

In [ ]:
descriptor = pd.DataFrame()
cnt = 0
print 'start : ' + time.strftime('%a %H:%M:%S')
for pair in desc_pair:
    query_drug = pair[0][0]
    query_disease = pair[0][1]
    query_asso = pair[0][2]
    pos_drug = pair[1][0]
    pos_disease = pair[1][1]
    if (query_drug == pos_drug) & (query_disease == pos_disease):
            continue
    else:
        found_drug = df_drug_sim[(df_drug_sim.DrugBank_ID1 == query_drug) & (df_drug_sim.DrugBank_ID2 == pos_drug)]
        drug_cheSim = found_drug['CheSim'].values[0]
        drug_pheSim = found_drug['PheSim'].values[0]
    
        found_disease = df_disease_sim[(df_disease_sim.Disease_ID1 == query_disease) & (df_disease_sim.Disease_ID2 == pos_disease)]
        disease_pheSim = found_disease['PheSim'].values[0]
        disease_geSim = found_disease['GeSim'].values[0]
    
        drug_sim = [drug_cheSim, drug_pheSim]
        disease_sim = [disease_geSim, disease_pheSim]
        
        sim = [math.sqrt(x[0]*x[1]) for x in list(product(drug_sim, disease_sim))]
        temp_df = pd.DataFrame(data = [[query_drug, query_disease, query_asso, sim[0], sim[1], sim[2], sim[3]]],
                           columns=['drug', 'disease', 'association', 'DrChe_DiGe', 'DrChe_DiPhe', 'DrPhe_DiGe', 'DrPhe_DiPhe'])
        descriptor = descriptor.append(temp_df)
        cnt = cnt +1
        if cnt % 1000 == 0:
            print "{} : ".format(cnt) + time.strftime('%a %H:%M:%S')
print 'end : ' + time.strftime('%a %H:%M:%S')

In [ ]:
descriptor = pd.DataFrame()
cnt = 0
print 'start : ' + time.strftime('%a %H:%M:%S')
bef_descriptor = list()
for index, row in new_asso.iterrows(): # all association
    print time.strftime('%a %H:%M:%S')
    query_drug = row['drug']
    query_disease = row['disease']
    query_asso = row['association']
    similarities = list()
    for index2, row2, in pos.iterrows(): # known association
        pos_drug = row2['drug']
        pos_disease = row2['disease']
        if (query_drug == pos_drug) & (query_disease == pos_disease):
            continue
        else:
            drug_cheSim = drug_ch[(drug_ch.DrugBank_ID1 == query_drug) & (drug_ch.DrugBank_ID2 == pos_drug)]['Similarity_Score']
            drug_pheSim = drug_ph[(drug_ph.DrugBank_ID1 == query_drug) & (drug_ph.DrugBank_ID2 == pos_drug)]['Similarity_Score']
            disease_pheSim = disease_ph[[str(query_disease)]].loc[pos_disease]
            disease_geSim = disease_ge[(disease_ge.Disease_ID1 == query_disease) & (disease_ge.Disease_ID2 == pos_disease)]['Similarity_Score']
            drug_sim = [drug_cheSim.values[0], drug_pheSim.values[0]]
            disease_sim = [disease_geSim.values[0], disease_pheSim.values[0]]
            similarities.append([drug_sim, disease_sim])
            #similarities.append([math.sqrt(x[0]*x[1]) for x in list(product(drug_sim, disease_sim))])
    bef_descriptor.append(similarities)    
    cnt = cnt + 1
    print time.strftime('%a %H:%M:%S')
    if cnt % 1000 == 0 :
        print "cnt : {} ".format(cnt) + time.strftime('%a %H:%M:%S')
print 'end : ' + time.strftime('%a %H:%M:%S')

In [ ]:
print query_disease
print pos_disease
print disease_ph[[str(query_disease)]].loc[pos_disease]

In [ ]:
cnt = 0
print time.strftime('%a %H:%M:%S')
bef_descriptor = list()
for index, row in new_asso.iterrows(): # all association
    query_drug = row['drug']
    query_disease = row['disease']
    query_asso = row['association']
    similarities = list()
    print time.strftime('%a %H:%M:%S')
    for index2, row2, in pos.iterrows(): # known association
        pos_drug = row2['drug']
        pos_disease = row2['disease']
        if (query_drug == pos_drug) & (query_disease == pos_disease):
            continue
        else:
            drug_cheSim = drug_ch[(drug_ch.DrugBank_ID1 == query_drug) & (drug_ch.DrugBank_ID2 == pos_drug)]['Similarity_Score']
            drug_pheSim = drug_ph[(drug_ph.DrugBank_ID1 == query_drug) & (drug_ph.DrugBank_ID2 == pos_drug)]['Similarity_Score']
            disease_pheSim = disease_ph[[str(query_disease)]].loc[pos_disease]
            disease_geSim = disease_ge[(disease_ge.Disease_ID1 == query_disease) & (disease_ge.Disease_ID2 == pos_disease)]['Similarity_Score']
            drug_sim = [drug_cheSim.values[0], drug_pheSim.values[0]]
            disease_sim = [disease_geSim.values[0], disease_pheSim.values[0]]
            similarities.append([math.sqrt(x[0]*x[1]) for x in list(product(drug_sim, disease_sim))])

    bef_descriptor.append(similarities)
    print time.strftime('%a %H:%M:%S')
    break
    cnt = cnt + 1
    if cnt % 1000 == 0 :
        print "cnt : {} ".format(cnt) + time.strftime('%a %H:%M:%S')
print time.strftime('%a %H:%M:%S')

In [ ]:
cnt